In [1]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 147.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 100.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
import open3d as o3d
import numpy as np
import json

In [3]:
bed = o3d.io.read_point_cloud("/content/pcd_bed_2.ply")
bookshelf = o3d.io.read_point_cloud("/content/pcd_bookshelf_2.ply")

In [4]:
bed.paint_uniform_color ([1 , 0.706 , 0])
bookshelf.paint_uniform_color ([0 , 0.651 , 0.929]) 

theta = np.radians(-90)
R = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])

t = np.array([2.0, 0.0, 0.0])

T = np.eye(4)
T[:3, :3] = R
T[:3, 3] = t

print("Transformation Matrix applied to Wall B:\n", T)

bed.transform ( T )
combined_pcd = bed + bookshelf

o3d.visualization.draw_geometries ([ combined_pcd ])
o3d.io.write_point_cloud ("merged_room_2.ply", combined_pcd )

Transformation Matrix applied to Wall B:
 [[ 6.123234e-17  0.000000e+00 -1.000000e+00  2.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00]
 [ 1.000000e+00  0.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
[Open3D WARNING] GLFW Error: Failed to detect any supported platform
[Open3D WARNING] GLFW initialized for headless rendering.
[Open3D WARNING] GLFW Error: OSMesa: Library not found
[Open3D WARNING] Failed to create window
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


True

In [5]:
np.save ("transformation_matrix.npy", T )

In [5]:
def transform_camera_pose(R_old, t_old, T):
    R_transform = T[:3, :3]
    t_transform = T[:3, 3]

    C_old = -R_old.T @ t_old
    R_new = R_transform @ R_old
    C_new = R_transform @ C_old + t_transform
    t_new = -R_new @ C_new

    return R_new, t_new


def transform_cameras(T):
    with open("/content/cameras_bed_2.json", 'r') as f:
        cameras = json.load(f)

    for cam in cameras['cameras']:
        R_old = np.array(cam['rotation'])
        t_old = np.array(cam['translation'])
        R_new, t_new = transform_camera_pose(R_old, t_old, T)
        cam['rotation'] = R_new.tolist()
        cam['translation'] = t_new.tolist()
        cam["scene"] = "bed"

    with open("cameras_bed_transformed.json", 'w') as f:
        json.dump(cameras, f, indent=2)


def combine_cameras():
    with open("/content/cameras_bookshelf_2.json", 'r') as f:
        cameras_A = json.load(f)
    with open("cameras_bed_transformed.json", 'r') as f:
        cameras_B = json.load(f)

    all_cameras = {
        "cameras": cameras_A['cameras'] + cameras_B['cameras']
    }

    for cam in cameras_A['cameras']:
      cam["scene"] = "bookshelf"

    with open("all_cameras_2.json", 'w') as f:
        json.dump(all_cameras, f, indent=2)



In [6]:
print("\nStep 2: Transforming cameras...")
transform_cameras(T)

print("\nStep 3: Combining all cameras...")
combine_cameras()

print("\n✅ All steps complete!")


Step 2: Transforming cameras...

Step 3: Combining all cameras...

✅ All steps complete!
